In [1]:
# step 4: stats file within each dataset. 

In [2]:
import pandas as pd
# clusterId is ENCSR618WVK_Cluster5
def getManualAnnotationLabel(clusterId,local_folder):
    print("getManualAnnotationLabel: clusterId is {}".format(clusterId))
    print("getManualAnnotationLabel: local_folder is {}".format(local_folder))
    if ("_" in clusterId):
        clusterId = clusterId.split("_")[1]
    id = clusterId.replace("Cluster","")
    print("getManualAnnotationLabel: id is {}".format(id))
    print("getManualAnnotationLabel: local_folder is {}".format(local_folder))
   
    cluster_id_annotation_map_file = os.listdir(local_folder)
    cluster_id_annotation_map_file = [x for x in cluster_id_annotation_map_file if "cluster_id_annotation_map" in x]
    print("getManualAnnotationLabel: cluster_id_annotation_map_file is {}".format(cluster_id_annotation_map_file))
    df = pd.read_csv(os.path.join(local_folder,cluster_id_annotation_map_file[0]), delimiter="\t")
    print("getManualAnnotationLabel:df is {}".format(df))
    # Filter the DataFrame to get rows where ClusterId is 2
    filtered_rows = df[df['ClusterId'] == int(id)]
    # Extract the 'manualAnnotationLabel' values from the filtered rows
    manual_annotation_labels = filtered_rows['ManualAnnotationLabel']
    Cell = ', '.join(manual_annotation_labels)
    print("getManualAnnotationLabel: Cell is {}".format(Cell))
    return(Cell)

In [3]:
# getCellClusterID: file is ENCSR618WVK_Cluster5.tsv.tagAlign

In [4]:
# "ENCSR618WVK_Cluster5.tsv.tagAlign".split(".tsv.tagAlign")[0]

In [5]:
def getCellClusterID(file,subdir):
    print("getCellClusterID: file is {}".format(file))
    # ClusterID=file.split(".tsv.tagAlign")[0].split("_")[1]
    ClusterID = file.split(".tsv.tagAlign")[0]
    print("ClusterID is {}".format(ClusterID))
    return(ClusterID)

In [6]:
def getMeanRNAUMIsPerCell():
    return()

In [7]:
import os
import pandas as pd
clusters_folder_path = "./clusters/"

# List all subdirectories under the "clusters" folder
subdirectories = [name for name in os.listdir(clusters_folder_path) if os.path.isdir(os.path.join(clusters_folder_path, name))]
subdirectories = [x for x in subdirectories if "backup" not in x and ".ipynb" not in x]

# Create an empty dictionary to store the files for each subdirectory
subdir_files_dict = {}

# List all files under each subdirectory and store in the dictionary
# for subdir in subdirectories[0:1]:  # for debug
for subdir in subdirectories:  # all files
    subdir_path = os.path.join(clusters_folder_path, subdir)
    files = [file for file in os.listdir(subdir_path) if os.path.isfile(os.path.join(subdir_path, file))]
    subdir_files_dict[subdir] = files

# Display the files for each subdirectory
for subdir, files in subdir_files_dict.items():
    print(f"Subdirectory: {subdir}")
    stats_output_file = os.path.join(clusters_folder_path,subdir,'{}_ClusterMetadata.tsv'.format(subdir))
    print("stats_output_file is {}".format(stats_output_file))
    if os.path.exists(stats_output_file):
        print("stats_output_file {} exists".format(stats_output_file))
        continue
        
    print("Files:")
    print(files)
    files = [x for x in files if x.endswith(".tsv.tagAlign")]
    files_in_subdir = len(files)
    print("files_in_subdir {} is {}".format(subdir,files_in_subdir))
    # Define the headers of the stats TSV file
    headers = ['tagAlignFile','CellClusterID','ManualAnnotationLabel','nCells','MeanRNAUMIsPerCell','MeanATACFragmentsPerCell']
    # Create an empty DataFrame with the headers
    df_stats = pd.DataFrame(columns=headers)
    # for index, file in enumerate(files[0:1]): # debug
    for index, file in enumerate(files): # all files
        print("file is {}".format(file))
        # Read the TSV file into a pandas DataFrame
        df = pd.read_csv(os.path.join(clusters_folder_path,subdir,file), delimiter='\t', \
                         names = ['chr','start','stop','cell_id','reads','strand'])
        # Now you can work with the DataFrame 'df'
        print(df.head(2))  # Example: display the first few rows of the DataFrame
        print("number of rows (+/-) is {}".format(df.shape[0]))
        # Filter out rows with "-" in the last column
        df_positive_strand = df[~(df.iloc[:, -1] == '-')]
        # Now you can work with the filtered DataFrame 'df_filtered'
        print(df_positive_strand.head(2))  # Example: display the first few rows of the filtered DataFrame
        number_of_fragments = df_positive_strand.shape[0]
        print("number_of_fragments = number of rows (+) is {}".format(number_of_fragments))
        # CellClusterID	ManualAnnotationLabel	nCells	MeanRNAUMIsPerCell	MeanATACFragmentsPerCell
        # Xu2020_Cluster1	CD4+ T cell		152	3500			5000
        CellClusterID = getCellClusterID(file,subdir)
        print("CellClusterID is {}".format(CellClusterID))
        dir_man = os.path.join(clusters_folder_path,subdir)
        print("dir_man is ",dir_man)
        ManualAnnotationLabel = getManualAnnotationLabel(CellClusterID,os.path.join(clusters_folder_path,subdir))
        print("ManualAnnotationLabel is {}".format(ManualAnnotationLabel))
        nCells = df_positive_strand['cell_id'].nunique()
        print("nCells is {}".format(nCells))
        MeanRNAUMIsPerCell = getMeanRNAUMIsPerCell()
        MeanATACFragmentsPerCell=""
        if (nCells!=0):
            MeanATACFragmentsPerCell = number_of_fragments // nCells
            print("MeanATACFragmentsPerCell is {}".format(MeanATACFragmentsPerCell))
        else:
            print("nCells is zero")
        data_to_Add = [[CellClusterID,ManualAnnotationLabel,nCells,"",MeanATACFragmentsPerCell]]
        # Row to add as a dictionary
        new_row = {'tagAlignFile': file,
                   'CellClusterID': CellClusterID, 
                   'ManualAnnotationLabel': ManualAnnotationLabel,
                   'nCells':nCells, 
                   'MeanRNAUMIsPerCell':MeanRNAUMIsPerCell,
                   'MeanATACFragmentsPerCell':MeanATACFragmentsPerCell}
        # Add the new row using loc
        df_stats.loc[len(df_stats)] = new_row
        print(df_stats.head())
    # Save the DataFrame to a TSV file for each dataset
    df_stats.to_csv(stats_output_file, sep='\t', index=False)


Subdirectory: ENCSR052DKH
stats_output_file is ./clusters/ENCSR052DKH/ENCSR052DKH_ClusterMetadata.tsv
stats_output_file ./clusters/ENCSR052DKH/ENCSR052DKH_ClusterMetadata.tsv exists
Subdirectory: ENCSR397LYX
stats_output_file is ./clusters/ENCSR397LYX/ENCSR397LYX_ClusterMetadata.tsv
stats_output_file ./clusters/ENCSR397LYX/ENCSR397LYX_ClusterMetadata.tsv exists
Subdirectory: ENCSR367GKP
stats_output_file is ./clusters/ENCSR367GKP/ENCSR367GKP_ClusterMetadata.tsv
stats_output_file ./clusters/ENCSR367GKP/ENCSR367GKP_ClusterMetadata.tsv exists
Subdirectory: ENCSR205TUH
stats_output_file is ./clusters/ENCSR205TUH/ENCSR205TUH_ClusterMetadata.tsv
stats_output_file ./clusters/ENCSR205TUH/ENCSR205TUH_ClusterMetadata.tsv exists
Subdirectory: ENCSR868ZEI
stats_output_file is ./clusters/ENCSR868ZEI/ENCSR868ZEI_ClusterMetadata.tsv
stats_output_file ./clusters/ENCSR868ZEI/ENCSR868ZEI_ClusterMetadata.tsv exists
Subdirectory: ENCSR506YMX
stats_output_file is ./clusters/ENCSR506YMX/ENCSR506YMX_ClusterM

In [8]:
# CellClusterID	ManualAnnotationLabel	nCells	MeanRNAUMIsPerCell	MeanATACFragmentsPerCell
# Xu2020_Cluster1	CD4+ T cell		152	3500			5000
# Xu2020_Cluster2	CD4+ T cell		357	5000			6000


In [9]:
# !ls Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/synapse_in_out/clusters/backup_ENCSR618WV*